In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# cd '/content/drive/Shared drives/CIS522'

Mounted at /content/drive


In [ ]:
!pip install -q contractions

import pandas as pd 
import nltk
import re
import matplotlib.pyplot as plt
import contractions


nltk.download('punkt')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 4.3 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Downloading Dataset

In [ ]:
# from google.colab import files

# # TODO: UPLOAD kaggle.json file - download it frm kaggle.com if you dont have
# files.upload() #this will prompt you to upload the kaggle.json


# !ls -lha kaggle.json
# !pip install -q kaggle

In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/

# # we need to set permissions
# !chmod 600 /root/.kaggle/kaggle.json

# # check your directory before downloading the datasets
# !pwd

# # list all available datasets
# !kaggle datasets list -s stackoverflow

# !kaggle datasets download -d michaelfumery/stackoverflow-questions-filtered-2011-2021

#### Loading in Dataset - combines 2009-2020

### Preprocessing

1) Creation date to datetime
2) Separate Tags 

3) Make text lowercase 
4) Remove punctuation
5) Remove tags + extract url
6) Remove stop words
7) Encode the text

8) Remove links/hyperlinks?
9) Remove html tags?
10) Remove whitespace
11) Remove consecutive '\n's


Feature engineering: 
12) Add post length

x) Tokenize the text


TODO / QUESTIONS?
* What type of puncutation should we remove?
* How to deal with URLs in post? If a post has same URL link that's a good thing. Also if a URL is another Stackoverflow post, what to do?
* How to deal with coding syntax?

In [ ]:
years = [i for i in range(2009, 2020)]


for year in years: 
  path = ("/content/drive/Shareddrives/CIS522/stackoverflow-questions-filtered-2011-2021/StackOverflow_questions_%d.csv" % year)
  if year == 2009:
    df = pd.read_csv(path)
  else: 
    df_new = pd.read_csv(path)
    df = pd.concat([df, df_new])

df.shape

(101611, 9)

In [ ]:
df.head()

,Id,CreationDate,Title,Body,Tags,ViewCount,CommentCount,AnswerCount,Score
0,414336,2009-01-05 20:09:49,Why does the SqlServer optimizer get so confus...,<p>I know this has something to do with parame...,<sql><sql-server><optimization>,3368,6,9,7
1,414605,2009-01-05 21:33:21,Fluent interface for rendering HTML,<p>Rendering HTML with the HtmlTextWriter isn'...,<.net><html><render><web-controls><htmltextwri...,4557,7,4,10
2,414714,2009-01-05 22:21:04,Compiling with g++ using multiple cores,<p>Quick question: what is the compiler flag t...,<c++><compiler-construction><makefile><g++><mu...,159848,7,8,181
3,415452,2009-01-06 04:26:16,Object-orientation in C,<p>What would be a set of nifty preprocessor h...,<c><oop><object>,82624,6,23,161
4,415515,2009-01-06 04:55:55,How can I read and manipulate CSV file data in...,"<p>Pretty self-explanatory, I tried google and...",<c++><csv>,147860,7,9,51


In [ ]:
num_posts = df.shape[0]

print("number of posts: %d" % num_posts)

number of posts: 101611


####1) Change to creation date to datetime

In [ ]:
# 1
df['CreationDate'] = pd.to_datetime(df['CreationDate'])

In [ ]:
df.head(2)

,Id,CreationDate,Title,Body,Tags,ViewCount,CommentCount,AnswerCount,Score
0,414336,2009-01-05 20:09:49,Why does the SqlServer optimizer get so confus...,<p>I know this has something to do with parame...,<sql><sql-server><optimization>,3368,6,9,7
1,414605,2009-01-05 21:33:21,Fluent interface for rendering HTML,<p>Rendering HTML with the HtmlTextWriter isn'...,<.net><html><render><web-controls><htmltextwri...,4557,7,4,10


####2) Parse tags 

In [ ]:
# 2
df['Tags'] = df['Tags'].apply(lambda tags_str: [tag.split(">")[0] for tag in tags_str.split("<")[1:]])

#### 3) Change body and title to lowercase 

In [ ]:
# 3
df['Title'] = df['Title'].str.lower()
df['Body'] = df['Body'].str.lower()

#### 3) Remove contractions

In [ ]:
def expand_contractions(text):
    text = contractions.fix(text)
    return text


In [ ]:
df['Title'] = df['Title'].apply(lambda x: expand_contractions(x)) 
df['Body'] = df['Body'].apply(lambda x: expand_contractions(x))

#### 4) Remove puncutation 

In [ ]:
# 4 - punc
df['Title']

0       why does the sqlserver optimizer get so confus...
1                     fluent interface for rendering html
2                 compiling with g++ using multiple cores
3                                 object-orientation in c
4       how can i read and manipulate csv file data in...
                              ...                        
3408    how to use npm package as normal javascript in...
3409    warning:tensorflow:sample_weight modes were co...
3410           trouble with cors policy and .net core 3.1
3411    how to abort only function on error, but do no...
3412                 file upload using foreach in laravel
Name: Title, Length: 101611, dtype: object

In [ ]:
df.columns

Index(['Id', 'CreationDate', 'Title', 'Body', 'Tags', 'ViewCount',
       'CommentCount', 'AnswerCount', 'Score'],
      dtype='object')

#### 5) remove html tags and extract URLs


In [ ]:
url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
href_pattern = r'<a href="([^"]+)">([^<]+)<\/a>'
p_pattern = r'<\/?p[^>]*>'


# extract urls
df['urls'] = df['Body'].apply(lambda x: re.findall(url_pattern, x)) 

# remove href tags + url 
df['Clean Body'] = df['Body'].apply(lambda text: re.sub(href_pattern, '', text))

# remove paragraph tags (<p>)
df['Clean Body'] = df['Clean Body'].apply(lambda text: re.sub(p_pattern, '', text))

df.head(10)

df['urls']

0       [https://stackoverflow.com/questions/379007/kn...
1                                                      []
2                                                      []
3       [http://www.planetpdf.com/codecuts/pdfs/ooc.pd...
4                                                      []
                              ...                        
3408    [https://github.com/niklasvh/html2canvas/relea...
3409    [https://github.com/tensorflow/tensorflow/blob...
3410    [https://localhost:8082/api/auth/', http://loc...
3411                                                   []
3412                                                   []
Name: urls, Length: 101611, dtype: object

In [ ]:
def show_clean_body(num_i):
  for i, x in enumerate(df['Clean Body']):
    if i == num_i:
      break
    print(i, x)

#### 6) Clean whitespace and new lines

In [ ]:
df['Clean Body'] = df['Clean Body'].apply(lambda text: text.replace("\n", " "))

####7) Remove code snippets in Body

In [ ]:
df['Clean Body'] = df['Clean Body'].apply(lambda text: re.sub('<.*?>', '', text))

In [ ]:
show_clean_body(5)

0 i know this has something to do with parameter sniffing, but i am just perplexed at how something like the following example is even possible with a piece of technology that does so many complex things well.  many of us have run into stored procedures that intermittently run several of orders of magnitude slower than usual, and then if you copy out the sql from the procedure and use the same parameter values in a separate query window, it runs as fast as usual.  i just fixed a procedure like that by converting this:  alter procedure p_myproc (     @param1 int ) as -- do a complex query with @param1   to this:  alter procedure p_myproc (     @param1 int ) as  declare @param1copy int; set @param1copy = @param1;  -- do the query using @param1copy   it went from running in over a minute back down to under one second, like it usually runs.  this behavior seems totally random.  for 9 out of 10 @param1 inputs, the query is fast, regardless of how much data it ends up needing to crunch, or h

### Feature Engineering 


#### 12) Add post length 

In [ ]:
df['body_length'] = df['Clean Body'].apply(lambda x: len(str(x)))

#### 13) Add number of code snippets 

In [ ]:
# Count the number of code snippets in each post
code_regex = r'<code>[\s\S]*?<\/code>'

df['code_snippets'] = df['Body'].apply(lambda x: len(re.findall(code_regex, str(x))))

In [ ]:
df['code_snippets'].head(5)

0    2
1    4
2    0
3    0
4    0
Name: code_snippets, dtype: int64

#### Tokenize text

#####Tokenize Title

In [ ]:
df['clean tokenized title'] = df['Title'].apply(lambda text: nltk.word_tokenize(text))

df['clean tokenized title'].head(5)

0    [why, does, the, sqlserver, optimizer, get, so...
1            [fluent, interface, for, rendering, html]
2       [compiling, with, g++, using, multiple, cores]
3                          [object-orientation, in, c]
4    [how, can, i, read, and, manipulate, csv, file...
Name: clean tokenized title, dtype: object

In [ ]:
df.head()

,Id,CreationDate,Title,Body,Tags,ViewCount,CommentCount,AnswerCount,Score,urls,Clean Body,body_length,code_snippets,clean tokenized title
0,414336,2009-01-05 20:09:49,why does the sqlserver optimizer get so confus...,<p>i know this has something to do with parame...,"[sql, sql-server, optimization]",3368,6,9,7,[https://stackoverflow.com/questions/379007/kn...,i know this has something to do with parameter...,1578,2,"[why, does, the, sqlserver, optimizer, get, so..."
1,414605,2009-01-05 21:33:21,fluent interface for rendering html,<p>rendering html with the htmltextwriter is n...,"[.net, html, render, web-controls, htmltextwri...",4557,7,4,10,[],rendering html with the htmltextwriter is not ...,5117,4,"[fluent, interface, for, rendering, html]"
2,414714,2009-01-05 22:21:04,compiling with g++ using multiple cores,<p>quick question: what is the compiler flag t...,"[c++, compiler-construction, makefile, g++, mu...",159848,7,8,181,[],quick question: what is the compiler flag to a...,197,0,"[compiling, with, g++, using, multiple, cores]"
3,415452,2009-01-06 04:26:16,object-orientation in c,<p>what would be a set of nifty preprocessor h...,"[c, oop, object]",82624,6,23,161,[http://www.planetpdf.com/codecuts/pdfs/ooc.pd...,what would be a set of nifty preprocessor hack...,477,0,"[object-orientation, in, c]"
4,415515,2009-01-06 04:55:55,how can i read and manipulate csv file data in...,"<p>pretty self-explanatory, i tried google and...","[c++, csv]",147860,7,9,51,[],"pretty self-explanatory, i tried google and go...",181,0,"[how, can, i, read, and, manipulate, csv, file..."


#####Tokenize Body (uncomment - takes 3-4 mins)



In [ ]:
df['clean tokenized body'] = df['Clean Body'].apply(lambda text: nltk.word_tokenize(text))

KeyboardInterrupt: ignored

In [ ]:
df.reset_index(inplace = True)

#### Create sets

In [ ]:
df['title set'] = df['clean tokenized title'].apply(lambda x: set(x))

In [ ]:
df['body set'] = df['clean tokenized title'].apply(lambda x: set(x))

In [ ]:
df['union body title set'] = df.apply(lambda x: x['title set'].union(x['body set']), axis=1)

In [ ]:
df = df.drop(columns = ['Title', 'Body', 'urls'])

In [ ]:
df.head()

,index,Id,CreationDate,Tags,ViewCount,CommentCount,AnswerCount,Score,Clean Body,body_length,code_snippets,clean tokenized title,clean tokenized body,title set,body set,union body title set
0,0,414336,2009-01-05 20:09:49,"[sql, sql-server, optimization]",3368,6,9,7,i know this has something to do with parameter...,1578,2,"[why, does, the, sqlserver, optimizer, get, so...","[i, know, this, has, something, to, do, with, ...","{the, optimizer, so, parameters, with, does, g...","{the, optimizer, so, parameters, with, does, g...","{the, optimizer, with, does, get, confused, ?,..."
1,1,414605,2009-01-05 21:33:21,"[.net, html, render, web-controls, htmltextwri...",4557,7,4,10,rendering html with the htmltextwriter is not ...,5117,4,"[fluent, interface, for, rendering, html]","[rendering, html, with, the, htmltextwriter, i...","{rendering, interface, for, html, fluent}","{rendering, interface, for, html, fluent}","{rendering, interface, for, html, fluent}"
2,2,414714,2009-01-05 22:21:04,"[c++, compiler-construction, makefile, g++, mu...",159848,7,8,181,quick question: what is the compiler flag to a...,197,0,"[compiling, with, g++, using, multiple, cores]","[quick, question, :, what, is, the, compiler, ...","{multiple, using, with, g++, compiling, cores}","{multiple, using, with, g++, compiling, cores}","{multiple, using, with, g++, compiling, cores}"
3,3,415452,2009-01-06 04:26:16,"[c, oop, object]",82624,6,23,161,what would be a set of nifty preprocessor hack...,477,0,"[object-orientation, in, c]","[what, would, be, a, set, of, nifty, preproces...","{in, object-orientation, c}","{in, object-orientation, c}","{object-orientation, in, c}"
4,4,415515,2009-01-06 04:55:55,"[c++, csv]",147860,7,9,51,"pretty self-explanatory, i tried google and go...",181,0,"[how, can, i, read, and, manipulate, csv, file...","[pretty, self-explanatory, ,, i, tried, google...","{read, file, and, manipulate, i, how, csv, dat...","{read, file, and, manipulate, i, how, csv, dat...","{read, and, i, how, can, ?, file, manipulate, ..."


In [ ]:
df = df.drop_duplicates(subset = 'Id')

In [ ]:
df.shape

(101611, 16)

In [ ]:
df.isna().sum()

index                    0
Id                       0
CreationDate             0
Tags                     0
ViewCount                0
CommentCount             0
AnswerCount              0
Score                    0
Clean Body               0
body_length              0
code_snippets            0
clean tokenized title    0
clean tokenized body     0
title set                0
body set                 0
union body title set     0
dtype: int64

##### Export to CSV

In [ ]:
cd '/content/drive/Shared drives/CIS522'

/content/drive/Shared drives/CIS522


In [ ]:
df.to_csv('final_final_df.csv', index = False)

For some really weird reason, when the df is saved to a csv it duplicates a ton of stuff and goes crazy so you have to run the function below to be able to use the cleaned df from the one saved.

In [ ]:
def fix_uploaded_csv(df):
  df = df.drop_duplicates(subset = 'Id')
  df = df.reset_index()
  df = df.drop(columns = ['index'])
  df = df.dropna(how = 'any')
  print(df.shape)
  return df

#### export to pkl

In [ ]:
df.head(5)

## EDA 

In [ ]:
df['Clean Body'].apply(lambda text: len(text))

0         1578
1         5117
2          197
3          477
4          181
          ... 
101606     692
101607     600
101608    2838
101609    1953
101610     631
Name: Clean Body, Length: 101611, dtype: int64

In [ ]:
print(df.iloc[0]["Title"])
print(df.iloc[0]["Clean Body"])

why does the sqlserver optimizer get so confused with parameters?
i know this has something to do with parameter sniffing, but i am just perplexed at how something like the following example is even possible with a piece of technology that does so many complex things well.  many of us have run into stored procedures that intermittently run several of orders of magnitude slower than usual, and then if you copy out the sql from the procedure and use the same parameter values in a separate query window, it runs as fast as usual.  i just fixed a procedure like that by converting this:  alter procedure p_myproc (     @param1 int ) as -- do a complex query with @param1   to this:  alter procedure p_myproc (     @param1 int ) as  declare @param1copy int; set @param1copy = @param1;  -- do the query using @param1copy   it went from running in over a minute back down to under one second, like it usually runs.  this behavior seems totally random.  for 9 out of 10 @param1 inputs, the query is fast

In [ ]:
print(df.iloc[1]["Title"])
print(df.iloc[1]["Clean Body"])

fluent interface for rendering html
rendering html with the htmltextwriter is not incredibly intuitive in my opinion, but if you are implementing web controls in web forms it is what you have to work with. i thought that it might be possible to create a fluent interface for this that reads a bit more like the html it outputs. i would like to know what people think of the syntax that i have come up with so far.      public void render(htmltextwriter writer)     {         writer             .tag(htmltextwritertag.div, e =&gt; e[htmltextwriterattribute.id, "id"][htmltextwriterattribute.name,"name"][htmltextwriterattribute.class,"class"])                 .tag(htmltextwritertag.span)                     .text("lorem")                 .endtag()                 .tag(htmltextwritertag.span)                     .text("ipsum")                 .endtag()             .endtag();             }   "tag", "text" and "endtag" are extension methods for the htmltextwriter class that returns the instance it

# Sample Pairs

In [ ]:
import numpy as np
samples = pd.read_pickle("samples_df.pkl")
#embedding = np.loadtxt('embedding.txt', dtype=float)

In [ ]:
from sklearn.model_selection import train_test_split
is_pos = samples['label'] == 'positive'

neg_samples = samples[~is_pos]
pos_samples = samples[is_pos]


pos_train_data, pos_test_data = train_test_split(pos_samples, test_size=0.2, random_state=42)

print("Positive Sample Train set size:", pos_train_data.shape)
print("Positive Sample Test set size:", pos_test_data.shape)

neg_train_data, neg_test_data = train_test_split(neg_samples, test_size=0.2, random_state=10)

print("Negative Sample Train set size:", neg_train_data.shape)
print("Negative Sample Test set size:", neg_test_data.shape)

# pd.DataFrame(embedding)
neg_train_data

Positive Sample Train set size: (4000, 3)
Positive Sample Test set size: (1000, 3)
Negative Sample Train set size: (4000, 3)
Negative Sample Test set size: (1000, 3)


,Pair,Score,label
2733,"(83955, 169)",-0.046819,negative
447,"(90425, 68231)",0.003197,negative
970,"(53685, 96521)",-0.076319,negative
125,"(35301, 60998)",0.018118,negative
798,"(54648, 94410)",0.039627,negative
...,...,...,...
1180,"(11788, 52929)",-0.032352,negative
3441,"(67359, 101044)",0.009788,negative
1344,"(2413, 18027)",-0.177910,negative
4623,"(71729, 16157)",0.007800,negative


# Gpt labelling

In [ ]:
%%capture
!pip install openai

In [ ]:
from getpass import getpass
import openai
import os

print('Enter OpenAI API key:')
openai.api_key = getpass()

os.environ['OPENAI_API_KEY']=openai.api_key

Enter OpenAI API key:
··········


In [ ]:
import openai
import os
import time

def generate_subject_few_shot(title1, body1, title2, body2):
  few_shot_prompt = """On a scale of 1 to 10, how similar are the following 2 questions? 
1. fluent interface for rendering html - 
rendering html with the htmltextwriter is not incredibly intuitive in my opinion, but if you are implementing web controls in web forms it is what you have to work with. i thought that it might be possible to create a fluent interface for this that reads a bit more like the html it outputs. i would like to know what people think of the syntax that i have come up with so far.      public void render(htmltextwriter writer)     {         writer             .tag(htmltextwritertag.div, e =&gt; e[htmltextwriterattribute.id, "id"][htmltextwriterattribute.name,"name"][htmltextwriterattribute.class,"class"])                 .tag(htmltextwritertag.span)                     .text("lorem")                 .endtag()                 .tag(htmltextwritertag.span)                     .text("ipsum")                 .endtag()             .endtag();             }   "tag", "text" and "endtag" are extension methods for the htmltextwriter class that returns the instance it takes in so that calls can be chained. the argument passed to the lambda used in the overload used by the first call to "tag" is a "htmlattributemanager", which is simple class that wraps an htmltextwriter to provide an indexer that takes an htmltextwriterattribute and a string value and returns the instance so that calls can be chained. i also have methods on this class for the most common attributes, such as "name", "class" and "id" so that you could write the first call above as follows:  .tag(htmltextwritertag.div, e =&gt; e.id("id").name("name").class("class"))   a little longer example:  public void render(htmltextwriter writer) {     writer         .tag(htmltextwritertag.div, a =&gt; a.class("someclass", "someotherclass"))             .tag(htmltextwritertag.h1).text("lorem").endtag()             .tag(htmltextwritertag.select, t =&gt; t.id("fooselect").name("fooselect").class("selectclass"))                 .tag(htmltextwritertag.option, t =&gt; t[htmltextwriterattribute.value, "1"][htmltextwriterattribute.title, "selects the number 1."])                     .text("1")                 .endtag(htmltextwritertag.option)                 .tag(htmltextwritertag.option, t =&gt; t[htmltextwriterattribute.value, "2"][htmltextwriterattribute.title, "selects the number 2."])                     .text("2")                 .endtag(htmltextwritertag.option)                 .tag(htmltextwritertag.option, t =&gt; t[htmltextwriterattribute.value, "3"][htmltextwriterattribute.title, "selects the number 3."])                     .text("3")                 .endtag(htmltextwritertag.option)             .endtag(htmltextwritertag.select)         .endtag(htmltextwritertag.div); }   hopefully you will be able to "decipher" what html this snippet outputs, at least that is the idea.  please give me any thoughts on how the syntax can be improved upon, maybe better method names, maybe some other approach all together.  edit: i thought it might be interesting to see what the same snippet would look like without the use of the fluent interface, for comparison:  public void renderusinghtmltextwriterstandardmethods(htmltextwriter writer) {     writer.addattribute(htmltextwriterattribute.class, "someclass someotherclass");     writer.renderbegintag(htmltextwritertag.div);      writer.renderbegintag(htmltextwritertag.h1);     writer.write("lorem");     writer.renderendtag();      writer.addattribute(htmltextwriterattribute.id, "fooselect");     writer.addattribute(htmltextwriterattribute.name, "fooselect");     writer.addattribute(htmltextwriterattribute.class, "selectclass");     writer.renderbegintag(htmltextwritertag.select);      writer.addattribute(htmltextwriterattribute.value, "1");     writer.addattribute(htmltextwriterattribute.title, "selects the number 1.");     writer.renderbegintag(htmltextwritertag.option);     writer.write("1");     writer.renderendtag();      writer.addattribute(htmltextwriterattribute.value, "2");     writer.addattribute(htmltextwriterattribute.title, "selects the number 2.");     writer.renderbegintag(htmltextwritertag.option);     writer.write("2");     writer.renderendtag();      writer.addattribute(htmltextwriterattribute.value, "3");     writer.addattribute(htmltextwriterattribute.title, "selects the number 3.");     writer.renderbegintag(htmltextwritertag.option);     writer.write("3");     writer.renderendtag();      writer.renderendtag();      writer.renderendtag(); }   edit: i should probably be a little more explicit in that one of the goals with this is that it should incur as little overhead as possible, this is why i have limited the use of lambdas. also at first i used a class that represented a tag so that something similar to a dom-tree was built by the syntax before the rendering, the syntax was very similar though. i abandoned this solution for the slight memory overhead it incurs. there are still some of this present in the use of the htmlattributemanager class, i have been thinking about using extension methods for the appending of attributes also, but the i cannot use the indexer-syntax, also it bloats the interface of the htmltextwriter even more. 
2. why does the sqlserver optimizer get so confused with parameters? - 
i know this has something to do with parameter sniffing, but i am just perplexed at how something like the following example is even possible with a piece of technology that does so many complex things well.  many of us have run into stored procedures that intermittently run several of orders of magnitude slower than usual, and then if you copy out the sql from the procedure and use the same parameter values in a separate query window, it runs as fast as usual.  i just fixed a procedure like that by converting this:  alter procedure p_myproc (     @param1 int ) as -- do a complex query with @param1   to this:  alter procedure p_myproc (     @param1 int ) as  declare @param1copy int; set @param1copy = @param1;  -- do the query using @param1copy   it went from running in over a minute back down to under one second, like it usually runs.  this behavior seems totally random.  for 9 out of 10 @param1 inputs, the query is fast, regardless of how much data it ends up needing to crunch, or how big the result set it.  but for that 1 out of 10, it just gets lost.  and the fix is to replace an int with the same int in the query?  it makes no sense.  [edit]  @gbn linked to this question, which details a similar problem:    i hesitate to cry "bug!" because that is so often a cop-out, but this really does seem like a bug to me.  when i run the two versions of my stored procedure with the same input, i see identical query plans.  the only difference is that the original takes more than a minute to run, and the version with the goofy parameter copying runs instantly. 

2

On a scale of 1 to 10, how similar are the following 2 questions? 
1. how do I concatenate two lists in Python? - 
how do I concatenate two lists in Python? Example: listone = [1, 2, 3] listtwo = [4, 5, 6] Expected outcome: >>> joinedlist [1, 2, 3, 4, 5, 6]
2. what is the fastest way to merge two lists in python? - 
Given, list_1 = [1,2,3,4] list_2 = [5,6,7,8] What is the fastest way to achieve the following in python? list = [1,2,3,4,5,6,7,8] Please note that there can be many ways to merge two lists in python. I am looking for the most time-efficient way.

9

  few_shot_prompt = """On a scale of 1 to 10, how similar are the following 2 questions? 
1. """

  few_shot_prompt += title1
  few_shot_prompt += """ - 
"""
  few_shot_prompt += body1
  few_shot_prompt += """
2. """
  few_shot_prompt += title2
  few_shot_prompt += """ - 
"""
  few_shot_prompt += body2
  few_shot_prompt += """
  
"""

  response = openai.Completion.create(
      model="text-davinci-003",
      prompt=few_shot_prompt, 
      temperature=0.0,
      max_tokens=1,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["\n"]
  )

  return response['choices'][0]['text'].strip()

In [ ]:
responses = []
for i in range(len(pos_test_data.index)):
  try: 
    x,y = pos_test_data.iloc[i]["Pair"]
    title1 = df.iloc[x]["Title"]
    body1 =  df.iloc[x]["Clean Body"]
    title2 = df.iloc[y]["Title"]
    body2 = df.iloc[y]["Clean Body"]
    response = generate_subject_few_shot(title1, body1, title2, body2)
    responses.append(response)
  except:
    responses.append("Invalid input")

print(responses)

['3', '3', '5', '2', '7', '5', '7', '7', '7', '7', '7', '3', '3', '3', '3', '7', '3', 'Invalid input', '3', '3', '7', '5', '7', '3', '3', '5', '7', '5', '3', '3', '5', '5', '3', '3', '7', '3', '3', '5', '3', '5', '3', '5', '7', '3', '5', '3', '5', '3', '3', '7', '5', '3', '7', '3', '5', '5', 'Invalid input', '5', '3', '7', '3', '5', '3', '3', '3', '5', '7', '5', '3', '7', '7', '3', '7', '3', '7', '3', 'Invalid input', '7', '7', '3', '7', '7', '3', '3', '7', '5', '', '3', '7', '5', '5', '2', '3', '3', '7', '3', '5', '5', '3', '7', '7', '3', '5', '3', '7', '3', '7', '3', '7', '3', '3', '5', 'Invalid input', '3', '7', '3', 'Invalid input', '3', '5', '3', '3', '7', '3', '7', '7', '5', '3', '3', '3', '7', '5', '7', '7', '7', 'Invalid input', '5', '7', '7', '5', '5', '7', '7', '3', '2', '7', '7', '3', '5', '7', '3', '3', '3', '7', '3', '3', '3', '3', '3', '5', '3', '7', '7', '7', '2', '5', '3', '5', '3', 'Invalid input', '7', '5', '7', '5', '3', '7', '7', '3', '3', '7', '3', '7', '7', '7', '

In [ ]:
pos_test_data["gpt_rating"] = responses
pos_test_data

,Pair,Score,label,gpt_rating
1501,"(35111, 49250)",0.691599,positive,3
2586,"(48244, 85640)",0.712601,positive,3
2653,"(71229, 14783)",0.641667,positive,5
1055,"(42101, 15042)",0.539217,positive,2
705,"(4748, 70103)",0.684667,positive,7
...,...,...,...,...
4711,"(90239, 69172)",0.605178,positive,7
2313,"(24596, 31426)",0.753308,positive,7
3214,"(51578, 6671)",0.683303,positive,5
2732,"(23472, 14134)",0.577998,positive,3


In [ ]:
positive_data = pos_test_data[pos_test_data['gpt_rating'] != 'Invalid input']
positive_data = positive_data[positive_data['gpt_rating'] != 'Answer']
positive_data = positive_data[positive_data['gpt_rating'] != 'Private']

In [ ]:
positive_data["gpt_rating"] = pd.to_numeric(positive_data["gpt_rating"])
positive_data.head()

,Pair,Score,label,gpt_rating
1501,"(35111, 49250)",0.691599,positive,3.0
2586,"(48244, 85640)",0.712601,positive,3.0
2653,"(71229, 14783)",0.641667,positive,5.0
1055,"(42101, 15042)",0.539217,positive,2.0
705,"(4748, 70103)",0.684667,positive,7.0


In [ ]:
positive_data["gpt_rating"].mean()

4.991587802313354

In [ ]:
positive_data.to_csv('positive_rated_samples.csv')

In [ ]:
responses = []
for i in range(len(neg_test_data.index)):
  try: 
    x,y = neg_test_data.iloc[i]["Pair"]
    title1 = df.iloc[x]["Title"]
    body1 =  df.iloc[x]["Clean Body"]
    title2 = df.iloc[y]["Title"]
    body2 = df.iloc[y]["Clean Body"]
    response = generate_subject_few_shot(title1, body1, title2, body2)
    responses.append(response)
  except:
    responses.append("Invalid input")

print(responses)

['2', '2', '1', '5', '3', '7', '5', '7', '3', '3', '7', '3', '3', '', '3', '3', '5', '2', '7', '7', '3', '7', '2', '3', '5', '7', '3', '3', '2', '7', '3', '3', '3', '7', '3', '7', '3', '3', '5', '3', '3', '7', '2', '2', '3', '3', '7', '7', '2', '7', '7', '3', '3', '7', '5', '2', '2', '7', '3', '7', '7', '3', '7', '3', '5', '3', 'Invalid input', '3', '3', '3', '3', '2', '2', '3', '3', '5', '3', 'Invalid input', '7', '3', '3', '7', '3', '5', '5', '3', '3', '3', '3', '5', '3', '7', '5', '5', '3', '3', '7', '3', '5', '1', '3', '5', '7', '3', '3', '3', '7', '3', '5', '3', '1', 'Invalid input', '5', '7', '7', '3', '3', '1', '3', '3', '3', '7', '3', '3', '3', '3', '2', '7', '3', '5', '3', '3', 'Invalid input', '7', '5', '1', '3', '3', '3', '3', '3', '7', '3', '3', '5', '5', '3', '7', '3', '3', '3', '7', '3', '2', '7', '3', '7', '3', '5', '7', '5', '5', '3', '5', '3', '7', '7', '3', '3', '3', '2', '5', '5', '3', '3', '3', '5', '3', '3', '3', '7', '3', '3', '7', '3', '3', '3', '5', '3', '5', '7

In [ ]:
neg_test_data["gpt_rating"] = responses
neg_test_data

,Pair,Score,label,gpt_rating
245,"(20319, 30685)",0.209722,negative,2
4493,"(32924, 95231)",0.027976,negative,2
4583,"(26670, 59665)",0.091165,negative,1
2242,"(46270, 41224)",-0.034114,negative,5
3407,"(12025, 95439)",0.043746,negative,3
...,...,...,...,...
877,"(48921, 64267)",-0.065769,negative,3
1314,"(97489, 35242)",0.088573,negative,1
2344,"(83133, 11184)",0.009395,negative,5
2717,"(39518, 99459)",0.295015,negative,5


In [ ]:
negative_data = neg_test_data[neg_test_data['gpt_rating'] != 'Invalid input']
negative_data = negative_data[negative_data['gpt_rating'] != 'Answer']
negative_data = negative_data[negative_data['gpt_rating'] != 'Private']
negative_data = negative_data[negative_data['gpt_rating'] != 'I']

In [ ]:
negative_data["gpt_rating"] = pd.to_numeric(negative_data["gpt_rating"])
negative_data.head()

,Pair,Score,label,gpt_rating
245,"(20319, 30685)",0.209722,negative,2.0
4493,"(32924, 95231)",0.027976,negative,2.0
4583,"(26670, 59665)",0.091165,negative,1.0
2242,"(46270, 41224)",-0.034114,negative,5.0
3407,"(12025, 95439)",0.043746,negative,3.0


In [ ]:
negative_data["gpt_rating"].mean()

4.094845360824742

In [ ]:
negative_data.to_csv('negative_rated_samples.csv')

## Bert dataset

In [ ]:
df1 = pd.read_csv('/content/positive_rated_samples.csv')
df2 = pd.read_csv('/content/negative_rated_samples.csv')

In [ ]:
df_merged = pd.concat([df1, df2], axis=0, join='inner')
df_merged = df_merged[["Pair", "gpt_rating"]]

In [ ]:
from ast import literal_eval
def s1(row):
  a, b = literal_eval(row["Pair"])
  text = df.iloc[a]["Title"] + '''
  ''' + df.iloc[a]["Clean Body"]
  return text

def s2(row):
  a, b = literal_eval(row["Pair"])
  text = df.iloc[b]["Title"] + '''
  ''' + df.iloc[b]["Clean Body"]
  return text

In [ ]:
df_merged['sentence1'] = df_merged.apply(lambda row : s1(row), axis = 1)
df_merged['sentence2'] = df_merged.apply(lambda row : s2(row), axis = 1)

In [ ]:
from sklearn.utils import shuffle
df_merged = shuffle(df_merged)
df_merged = df_merged.drop('Pair', axis=1)
df_merged = df_merged.reset_index()
df_merged = df_merged.reset_index()
df_merged = df_merged.drop('index', axis=1)
df_merged = df_merged.rename({'gpt_rating': 'label', 'level_0': 'idx'}, axis=1) 

In [ ]:
df_merged

,idx,label,sentence1,sentence2
0,0,3.0,bootstrap styling rounded corners\n i do not ...,how to create a triangle in css3 using border-...
1,1,7.0,headless chrome is not redirecting\n we are u...,chromedriver opens sites only with valid ssl c...
2,2,3.0,linux syscalls and errno\n context: i am tryi...,virtual function table of multiple inheritance...
3,3,7.0,how to save user input with js and show info w...,how can i start a windows app background task ...
4,4,2.0,"function with a custom return type and the ""fa...",is inline assembly language slower than native...
...,...,...,...,...
1936,1936,5.0,setting schema for all queries of a connection...,change postgres search_path with ecto\n i wou...
1937,1937,3.0,'dict' object has no attribute 'append' json\n...,"create input type=""button"" with two lines of t..."
1938,1938,3.0,dagger 2 examples\n dagger 2 is around the co...,changing frequency using fft and ifft not usin...
1939,1939,5.0,can i render a form from another website\n i ...,how to send password securely over http?\n if...


In [ ]:
df_merged.to_csv('bert_data.csv')